<a href="https://colab.research.google.com/github/RegiRau/Vein_Segmentation/blob/main/816x608Vein_Segmentation_Sweeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install wandb

In [ ]:
import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/816x608VeinSegmentation/Vein_Dataset_816x608.zip -d /content/

Archive:  /content/gdrive/MyDrive/816x608VeinSegmentation/Vein_Dataset_816x608.zip
   creating: /content/Vein_Dataset_816x608/images/
  inflating: /content/Vein_Dataset_816x608/images/001_image01_L.png  
  inflating: /content/Vein_Dataset_816x608/images/001_image01_R.png  
  inflating: /content/Vein_Dataset_816x608/images/001_image02_L.png  
  inflating: /content/Vein_Dataset_816x608/images/001_image02_R.png  
  inflating: /content/Vein_Dataset_816x608/images/001_image03_L.png  
  inflating: /content/Vein_Dataset_816x608/images/001_image03_R.png  
  inflating: /content/Vein_Dataset_816x608/images/001_image04_L.png  
  inflating: /content/Vein_Dataset_816x608/images/001_image04_R.png  
  inflating: /content/Vein_Dataset_816x608/images/001_image05_L.png  
  inflating: /content/Vein_Dataset_816x608/images/001_image05_R.png  
  inflating: /content/Vein_Dataset_816x608/images/001_image06_L.png  
  inflating: /content/Vein_Dataset_816x608/images/001_image06_R.png  
  inflating: /content/Vein

In [ ]:
import tensorflow as tf

def load_data():
  dataset_images = tf.data
  dataset_labels = tf.data
  SIZE_X = 816
  SIZE_Y = 608


  dataset_images = tf.keras.utils.image_dataset_from_directory("Vein_Dataset_816x608/images", labels=None, label_mode=None, color_mode="grayscale", batch_size=1, image_size=(SIZE_Y, SIZE_X), shuffle=False, )
  dataset_labels = tf.keras.utils.image_dataset_from_directory("Vein_Dataset_816x608/labels", labels=None, label_mode=None, color_mode="grayscale", batch_size=1, image_size=(SIZE_Y, SIZE_X), shuffle=False, )
  normalization_layer = tf.keras.layers.Rescaling(1./255)
  dataset_labels = dataset_labels.map(lambda x: (normalization_layer(x)))
  dataset_images = dataset_images.map(lambda x: (x /255.))

  Vein_Dataset = tf.data.Dataset.zip((dataset_images, dataset_labels))

  Vein_Dataset = Vein_Dataset.shuffle(buffer_size = 1000)

  Dataset_Size = 900

  train_size = int(0.7 * Dataset_Size)
  val_size = int(0.15 * Dataset_Size)
  test_size = int(0.15 * Dataset_Size)

  train_dataset = Vein_Dataset.take(train_size)
  test_dataset = Vein_Dataset.skip(train_size)
  val_dataset = test_dataset.skip(test_size)
  test_dataset = test_dataset.take(test_size)

  return train_dataset, val_dataset


In [ ]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K


#convolutional block
def conv_block(x, kernelsize, filters, dropout, batchnorm=False): 
    conv = layers.Conv2D(filters, (kernelsize, kernelsize), kernel_initializer='he_normal', padding="same")(x)
    if batchnorm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)
    conv = layers.Conv2D(filters, (kernelsize, kernelsize), kernel_initializer='he_normal', padding="same")(conv)
    if batchnorm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    return conv

        
    #skip connection    
    shortcut = layers.Conv2D(filters, kernel_size=(1, 1), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        shortcut = layers.BatchNormalization(axis=3)(shortcut)
    shortcut = layers.Activation("relu")(shortcut)
    respath = layers.add([shortcut, conv2])       
    return respath


#gating signal for attention unit
def gatingsignal(input, out_size, batchnorm=False):
    x = layers.Conv2D(out_size, (1, 1), padding='same')(input)
    if batchnorm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

#attention unit/block based on soft attention
def attention_block(x, gating, inter_shape):
    shape_x = K.int_shape(x)
    shape_g = K.int_shape(gating)
    theta_x = layers.Conv2D(inter_shape, (2, 2), strides=(2, 2), kernel_initializer='he_normal', padding='same')(x) 
    shape_theta_x = K.int_shape(theta_x)
    phi_g = layers.Conv2D(inter_shape, (1, 1), kernel_initializer='he_normal', padding='same')(gating)
    upsample_g = layers.Conv2DTranspose(inter_shape, (3, 3), strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]), kernel_initializer='he_normal', padding='same')(phi_g)
    concat_xg = layers.add([upsample_g, theta_x])
    act_xg = layers.Activation('relu')(concat_xg)
    psi = layers.Conv2D(1, (1, 1), kernel_initializer='he_normal', padding='same')(act_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = layers.UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg) 
    upsample_psi = layers.Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': shape_x[3]})(upsample_psi)                          
    y = layers.multiply([upsample_psi, x])
    result = layers.Conv2D(shape_x[3], (1, 1), kernel_initializer='he_normal', padding='same')(y)
    attenblock = layers.BatchNormalization()(result)
    return attenblock

#Attention U-NET
def attentionunet(input_shape, dropout=0.2, batchnorm=True):
    
    filters = [16, 32, 64, 128, 256]
    kernelsize = 3
    upsample_size = 2

    inputs = layers.Input(input_shape) 

    # Downsampling layers    
    dn_1 = conv_block(inputs, kernelsize, filters[0], dropout, batchnorm)
    pool_1 = layers.MaxPooling2D(pool_size=(2,2))(dn_1)
    
    dn_2 = conv_block(pool_1, kernelsize, filters[1], dropout, batchnorm)
    pool_2 = layers.MaxPooling2D(pool_size=(2,2))(dn_2)
    
    dn_3 = conv_block(pool_2, kernelsize, filters[2], dropout, batchnorm)
    pool_3 = layers.MaxPooling2D(pool_size=(2,2))(dn_3)
    
    dn_4 = conv_block(pool_3, kernelsize, filters[3], dropout, batchnorm)
    pool_4 = layers.MaxPooling2D(pool_size=(2,2))(dn_4)
    
    dn_5 = conv_block(pool_4, kernelsize, filters[4], dropout, batchnorm)

    # Upsampling layers    
    gating_5 = gatingsignal(dn_5, filters[3], batchnorm)
    att_5 = attention_block(dn_4, gating_5, filters[3])
    up_5 = layers.UpSampling2D(size=(upsample_size, upsample_size), data_format="channels_last")(dn_5)
    up_5 = layers.concatenate([up_5, att_5], axis=3)
    up_conv_5 = conv_block(up_5, kernelsize, filters[3], dropout, batchnorm)
    
    gating_4 = gatingsignal(up_conv_5, filters[2], batchnorm)
    att_4 = attention_block(dn_3, gating_4, filters[2])
    up_4 = layers.UpSampling2D(size=(upsample_size, upsample_size), data_format="channels_last")(up_conv_5)
    up_4 = layers.concatenate([up_4, att_4], axis=3)
    up_conv_4 = conv_block(up_4, kernelsize, filters[2], dropout, batchnorm)
   
    gating_3 = gatingsignal(up_conv_4, filters[1], batchnorm)
    att_3 = attention_block(dn_2, gating_3, filters[1])
    up_3 = layers.UpSampling2D(size=(upsample_size, upsample_size), data_format="channels_last")(up_conv_4)
    up_3 = layers.concatenate([up_3, att_3], axis=3)
    up_conv_3 = conv_block(up_3, kernelsize, filters[1], dropout, batchnorm)
    
    gating_2 = gatingsignal(up_conv_3, filters[0], batchnorm)
    att_2 = attention_block(dn_1, gating_2, filters[0])
    up_2 = layers.UpSampling2D(size=(upsample_size, upsample_size), data_format="channels_last")(up_conv_3)
    up_2 = layers.concatenate([up_2, att_2], axis=3)
    up_conv_2 = conv_block(up_2, kernelsize, filters[0], dropout, batchnorm)
    
    conv_final = layers.Conv2D(1, kernel_size=(1,1))(up_conv_2)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    outputs = layers.Activation('sigmoid')(conv_final)  

    model = models.Model(inputs=[inputs], outputs=[outputs])
    #model.summary()       
    return model    

In [ ]:
from tensorflow.keras import backend as K
from sklearn.metrics import jaccard_score,confusion_matrix


def IoU_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def IoU_loss(y_true, y_pred):
    return -IoU_coef(y_true, y_pred)
 
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)
 
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
  
def accuracy(y_true, y_pred):
    cm = confusion_matrix(y_true.flatten(),y_pred.flatten(), labels=[0, 1])
    acc = (cm[0,0]+cm[1,1])/(cm[0,0]+cm[0,1]+cm[1,0]+cm[1,1])
    return acc
  
def IoU(y_true, y_pred, labels = [0, 1]):
   IoU = []
   for label in labels:
      jaccard = jaccard_score(y_pred.flatten(),y_true.flatten(), pos_label=label, average='binary')
      IoU.append(jaccard)     
   return np.mean(IoU) 

In [ ]:
def Model(optim, drop):
  tf.keras.backend.clear_session()

  IMG_CHANNELS = 1
  SIZE_X = 816
  SIZE_Y = 608
  IMG_HEIGHT = SIZE_Y
  IMG_WIDTH = SIZE_X
  input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

  model = attentionunet(input_shape, dropout=drop)

  model.compile(optimizer = optim, loss= IoU_loss, metrics= ['accuracy', IoU_coef])

  return model

def train_step(x, y, model, optimizer, loss_fn, train_acc_metric):
  with tf.GradientTape() as tape:
    logits = model(x, training=True)
    loss_value = loss_fn(y, logits)
  
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    #train_acc_metric.update_state(y, logits)
    acc_metric_value = train_acc_metric(y, logits)

    return loss_value, acc_metric_value


def test_step(x, y, model, loss_fn, val_acc_metric):
    val_logits = model(x, training=False)
    loss_value = loss_fn(y, val_logits)
    #val_acc_metric.update_state(y, val_logits)
    val_acc_metric_value = val_acc_metric(y, val_logits)

    return loss_value, val_acc_metric_value

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

def build_optimizer(optim, learning_rate):
    if optim == "sgd":
        optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
    elif optim == "adam":
        optimizer = Adam(learning_rate=learning_rate)

    return optimizer

def build_loss(loss_function):
  if loss_function == "IoU_loss":
    loss = IoU_loss
  if loss_function == "BCE":
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)

  return loss


In [ ]:
def train(train_dataset,
          val_dataset, 
          model,
          optimizer,
          loss_fn,
          train_acc_metric,
          val_acc_metric,
          epochs=10, 
          log_step=200, 
          val_log_step=50):
  
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

        train_loss = []  
        train_acc = [] 
        val_loss = []
        val_acc = []

        # Iterate over the batches of the dataset
        for step, (x_batch_train, y_batch_train) in tqdm.tqdm(enumerate(train_dataset), total=len(train_dataset)):
            loss_value, acc_metric_value = train_step(x_batch_train, y_batch_train, 
                                    model, optimizer, 
                                    loss_fn, train_acc_metric)
            train_loss.append(float(loss_value))
            train_acc.append(float(acc_metric_value))

        # Run a validation loop at the end of each epoch
        for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
            val_loss_value, val_acc_metric_value = test_step(x_batch_val, y_batch_val, 
                                       model, loss_fn, 
                                       val_acc_metric)
            val_loss.append(float(val_loss_value))
            val_acc.append(float(val_acc_metric_value))
            
        # Display metrics at the end of each epoch
        #train_acc = train_acc_metric.result()
        train_IoU = np.mean(train_acc)
        #print("Training acc over epoch: %.4f" % (float(train_acc),))
        print("Training IoU over epoch: ", train_IoU,)

        #val_acc = val_acc_metric.result()
        val_IoU = np.mean(val_acc)
        #print("Validation acc: %.4f" % (float(val_acc),))
        print("Validation IoU over epoch: ", val_IoU)

        # # Reset metrics at the end of each epoch
        # train_acc_metric.reset_states()
        # val_acc_metric.reset_states()

        # 3️⃣ log metrics using wandb.log
        wandb.log({'epochs': epoch,
                   'loss': np.mean(train_loss),
                   'acc': train_IoU, #float(train_acc), 
                   'val_loss': np.mean(val_loss),
                   'val_acc': val_IoU }) #float(val_acc)})

In [ ]:
sweep_config = {
  'method': 'random', 
  'metric': {
      'name': 'val_loss', #'loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'batch_size': {
          'distribution': 'uniform',
          'min': 1,
          'max': 20
          #'values': [3, 32, 64, 128, 256]
      },
      'learning_rate':{
          # 'distribution': 'uniform',
          # 'min': 0.003,
          # 'max': 0.02
          'values': [0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 
                     0.01,  0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
      }#,
      # 'dropout': {
      #     'distribution':'uniform',
      #     'min': 0.0,
      #     'max': 0.1
          #'values': [0.05, 0.1]
      #},
      # 'loss': {
      #     'values': ['BCE'] #'IoU_loss', 
      #}#,

      # 'optimizer': {
      #   'values': ['adam', 'sgd']
      # },
  }
}

In [ ]:
from tensorflow.keras.optimizers import Adam

def sweep_train(config_defaults=None):
    # Set default values
    config_defaults = {
        "batch_size": 16, #wandb.config.batch_size, #16, #64,
        "learning_rate": 0.01
    }
    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)  # this gets over-written in the Sweep

    # Specify the other hyperparameters to the configuration, if any
    wandb.config.epochs = 5
    wandb.config.log_step = 20
    wandb.config.val_log_step = 50
    #wandb.config.architecture_name = "MLP"
    wandb.config.dataset_name = "Vein_Dataset"

    train_dataset, val_dataset = load_data()

    # # build input pipeline using tf.data
    # train_dataset = (train_dataset.shuffle(buffer_size=1024)
    #                               .batch(wandb.config.batch_size)
    #                               .prefetch(buffer_size=tf.data.AUTOTUNE))

    # val_dataset = (val_dataset.batch(wandb.config.batch_size)
    #                           .prefetch(buffer_size=tf.data.AUTOTUNE))

    # Instantiate an optimizer to train the model.
    #optimizer = build_optimizer(wandb.config.optimizer, wandb.config.learning_rate)
    optimizer = Adam(learning_rate=wandb.config.learning_rate)
    #optimizer = Adam(learning_rate= 0.001)
    #optimizer = keras.optimizers.SGD(learning_rate=wandb.config.learning_rate)

    # initialize model
    model = Model(optimizer, 0.0) #wandb.config.dropout)
    #model = Model(optimizer, 0.1)


    
    # Instantiate a loss function.
    #loss_fn = IoU_loss
    #loss_fn = build_loss(wandb.config.loss)
    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

    # Prepare the metrics.
    #train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
    train_acc_metric = IoU_coef
    #val_acc_metric = keras.metrics.SparseCategoricalAccuracy()
    val_acc_metric = IoU_coef

    train(train_dataset,
          val_dataset, 
          model,
          optimizer,
          loss_fn,
          train_acc_metric,
          val_acc_metric,
          epochs=wandb.config.epochs, 
          log_step=wandb.config.log_step, 
          val_log_step=wandb.config.val_log_step)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="816x608Sweeps")

Create sweep with ID: oh8kmjyk
Sweep URL: https://wandb.ai/regirau/816x608Sweeps/sweeps/oh8kmjyk


In [ ]:
wandb.agent(sweep_id, function=sweep_train, count=400)

wandb: Agent Starting Run: 5gzr2p0n with config:
wandb: 	batch_size: 18.858742176088978
wandb: 	learning_rate: 0.07
wandb: Currently logged in as: regirau. Use `wandb login --relogin` to force relogin


Found 900 files belonging to 1 classes.
Found 900 files belonging to 1 classes.

Start of epoch 0


100%|██████████| 630/630 [01:59<00:00,  5.29it/s]


Training IoU over epoch:  0.1658967733767534
Validation IoU over epoch:  0.16932337167672812

Start of epoch 1


100%|██████████| 630/630 [01:43<00:00,  6.10it/s]


...

wandb: Agent Starting Run: pmt6matw with config:
wandb: 	batch_size: 11.159283349600996
wandb: 	learning_rate: 0.005


Found 900 files belonging to 1 classes.
Found 900 files belonging to 1 classes.

Start of epoch 0


100%|██████████| 630/630 [01:43<00:00,  6.07it/s]


Training IoU over epoch:  0.1178322509936397
Validation IoU over epoch:  0.18017905375195875

Start of epoch 1


 88%|████████▊ | 552/630 [01:30<00:12,  6.22it/s]

In [ ]:
wandb.finish()